In [5]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
from pathlib import Path
import yaml
import time
import random
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, ssd300_vgg16
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
from torchvision import transforms


In [6]:
def preprocess_image_rcnn_ssd(img, input_size=(640, 640)):
    """
    Preprocess cho Faster R-CNN và SSD
    Trả về tensor đã chuẩn hóa, ratio và padding
    """
    orig_h, orig_w = img.shape[:2]
    target_h, target_w = input_size
    
    # Tính ratio để giữ aspect ratio
    ratio = min(target_w / orig_w, target_h / orig_h)
    new_w = int(orig_w * ratio)
    new_h = int(orig_h * ratio)
    
    # Resize ảnh
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    
    # Padding để đạt đúng input_size
    pad_w = target_w - new_w
    pad_h = target_h - new_h
    top = pad_h // 2
    bottom = pad_h - top
    left = pad_w // 2
    right = pad_w - left
    
    padded = cv2.copyMakeBorder(
        resized, top, bottom, left, right, 
        cv2.BORDER_CONSTANT, value=(114, 114, 114)
    )
    
    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(padded, cv2.COLOR_BGR2RGB)
    
    # Convert to tensor và normalize [0, 1]
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    img_tensor = transform(Image.fromarray(img_rgb))
    
    pad = (left, top)
    
    return img_tensor, ratio, pad


def postprocess_rcnn_ssd(predictions, orig_shape, ratio, pad, conf_threshold=0.25):
    """
    Postprocess cho Faster R-CNN và SSD
    predictions: dict với keys 'boxes', 'scores', 'labels'
    """
    boxes = predictions['boxes'].cpu().numpy()
    scores = predictions['scores'].cpu().numpy()
    labels = predictions['labels'].cpu().numpy()
    
    # Filter theo confidence threshold
    keep = scores >= conf_threshold
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]
    
    if len(boxes) == 0:
        return np.array([]), np.array([]), np.array([])
    
    # Unpad và scale về original size
    pad_left, pad_top = pad
    orig_h, orig_w = orig_shape
    
    # Remove padding
    boxes[:, [0, 2]] -= pad_left
    boxes[:, [1, 3]] -= pad_top
    
    # Scale về original size
    boxes[:, [0, 2]] /= ratio
    boxes[:, [1, 3]] /= ratio
    
    # Clip về image bounds
    boxes[:, [0, 2]] = np.clip(boxes[:, [0, 2]], 0, orig_w)
    boxes[:, [1, 3]] = np.clip(boxes[:, [1, 3]], 0, orig_h)
    
    # Convert labels (torchvision: 0=background, 1=traffic_sign)
    # Chuyển về 0-indexed cho class_ids
    class_ids = labels - 1
    
    return boxes, scores, class_ids


def benchmark_faster_rcnn_inference(
    model_path,
    data_yaml,
    model_name,
    input_size=(640, 640),
    num_warmup=5,
    num_samples=50,
    conf_threshold=0.25,
    device='cpu'
):
    """Benchmark Faster R-CNN với cùng preprocessing/postprocessing"""
    print(f"\n🚀 Benchmarking Faster R-CNN: {Path(model_path).name}")
    print(f"📐 Input size: {input_size}")
    print(f"🖥️  Device: {device}")
    print(f"📊 Running {num_samples} samples (after {num_warmup} warmup)\n")
    
    # Load model
    NUM_CLASSES = 2  # background + traffic_sign
    model = fasterrcnn_resnet50_fpn(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    
    # Load dataset
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    dataset_root = Path(data_yaml).parent
    val_path = dataset_root / data_config.get('val', 'valid/images')
    image_files = list(val_path.glob('*.jpg')) + list(val_path.glob('*.png'))
    
    if len(image_files) == 0:
        raise ValueError(f"No images found in {val_path}")
    
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
        
    # Warmup
    print(f"🔥 Warmup ({num_warmup} runs)...")
    for _ in range(num_warmup):
        img = cv2.imread(str(random.choice(sample_images)))
        input_tensor, _, _ = preprocess_image_rcnn_ssd(img, input_size=input_size)
        input_tensor = input_tensor.to(device)
        with torch.no_grad():
            _ = model([input_tensor])
    
    if device == 'cuda':
        torch.cuda.synchronize()
    
    # Benchmark
    preprocess_times, inference_times, postprocess_times = [], [], []
    
    print(f"\n⏱️  Running inference on {len(sample_images)} test images...")
    for img_path in tqdm(sample_images, desc="Processing images"):
        img = cv2.imread(str(img_path))
        orig_h, orig_w = img.shape[:2]
        
        # Preprocess
        t0 = time.time()
        input_tensor, ratio, pad = preprocess_image_rcnn_ssd(img, input_size=input_size)
        input_tensor = input_tensor.to(device)
        t1 = time.time()
        
        # Inference
        with torch.no_grad():
            predictions = model([input_tensor])
        
        if device == 'cuda':
            torch.cuda.synchronize()
        t2 = time.time()
        
        # Postprocess
        boxes, scores, class_ids = postprocess_rcnn_ssd(
            predictions[0], (orig_h, orig_w), ratio, pad, conf_threshold
        )
        t3 = time.time()
        
        preprocess_times.append(t1 - t0)
        inference_times.append(t2 - t1)
        postprocess_times.append(t3 - t2)
    
    # Summary
    avg_pre = np.mean(preprocess_times) * 1000
    avg_inf = np.mean(inference_times) * 1000
    avg_post = np.mean(postprocess_times) * 1000
    total_ms = avg_pre + avg_inf + avg_post
    fps = 1000 / total_ms
    
    print(f"\n📊 Average Timing (over {len(sample_images)} images):")
    print(f"   🧩 Preprocess: {avg_pre:.2f} ms")
    print(f"   ⚙️  Inference : {avg_inf:.2f} ms")
    print(f"   📦 Postprocess: {avg_post:.2f} ms")
    print(f"   ⏱️  Total: {total_ms:.2f} ms → {fps:.2f} FPS")
    
    return {
        "Model": model_name,
        "Backend": f"PyTorch Faster-RCNN ({device.upper()})",
        "Avg Preprocess (ms)": avg_pre,
        "Avg Inference (ms)": avg_inf,
        "Avg Postprocess (ms)": avg_post,
        "Total (ms)": total_ms,
        "FPS": fps,
    }

In [7]:

def benchmark_ssd300_inference(
    model_path,
    data_yaml,
    model_name,
    input_size=(640, 640),
    num_warmup=5,
    num_samples=50,
    conf_threshold=0.25,
    device='cpu'
):
    """Benchmark SSD300 với cùng preprocessing/postprocessing"""
    print(f"\n🚀 Benchmarking SSD300: {Path(model_path).name}")
    print(f"📐 Input size: {input_size}")
    print(f"🖥️  Device: {device}")
    print(f"📊 Running {num_samples} samples (after {num_warmup} warmup)\n")
    
    # Load model
    NUM_CLASSES = 2  # background + traffic_sign
    model = ssd300_vgg16(weights=None, num_classes=NUM_CLASSES)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    
    # Load dataset
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    dataset_root = Path(data_yaml).parent
    val_path = dataset_root / data_config.get('val', 'valid/images')
    image_files = list(val_path.glob('*.jpg')) + list(val_path.glob('*.png'))
    
    if len(image_files) == 0:
        raise ValueError(f"No images found in {val_path}")
    
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
        
    # Warmup
    print(f"🔥 Warmup ({num_warmup} runs)...")
    for _ in range(num_warmup):
        img = cv2.imread(str(random.choice(sample_images)))
        input_tensor, _, _ = preprocess_image_rcnn_ssd(img, input_size=input_size)
        input_tensor = input_tensor.to(device)
        with torch.no_grad():
            _ = model([input_tensor])
    
    if device == 'cuda':
        torch.cuda.synchronize()
    
    # Benchmark
    preprocess_times, inference_times, postprocess_times = [], [], []
    
    print(f"\n⏱️  Running inference on {len(sample_images)} test images...")
    for img_path in tqdm(sample_images, desc="Processing images"):
        img = cv2.imread(str(img_path))
        orig_h, orig_w = img.shape[:2]
        
        # Preprocess
        t0 = time.time()
        input_tensor, ratio, pad = preprocess_image_rcnn_ssd(img, input_size=input_size)
        input_tensor = input_tensor.to(device)
        t1 = time.time()
        
        # Inference
        with torch.no_grad():
            predictions = model([input_tensor])
        
        if device == 'cuda':
            torch.cuda.synchronize()
        t2 = time.time()
        
        # Postprocess
        boxes, scores, class_ids = postprocess_rcnn_ssd(
            predictions[0], (orig_h, orig_w), ratio, pad, conf_threshold
        )
        t3 = time.time()
        
        preprocess_times.append(t1 - t0)
        inference_times.append(t2 - t1)
        postprocess_times.append(t3 - t2)
    
    # Summary
    avg_pre = np.mean(preprocess_times) * 1000
    avg_inf = np.mean(inference_times) * 1000
    avg_post = np.mean(postprocess_times) * 1000
    total_ms = avg_pre + avg_inf + avg_post
    fps = 1000 / total_ms
    
    print(f"\n📊 Average Timing (over {len(sample_images)} images):")
    print(f"   🧩 Preprocess: {avg_pre:.2f} ms")
    print(f"   ⚙️  Inference : {avg_inf:.2f} ms")
    print(f"   📦 Postprocess: {avg_post:.2f} ms")
    print(f"   ⏱️  Total: {total_ms:.2f} ms → {fps:.2f} FPS")
    
    return {
        "Model": model_name,
        "Backend": f"PyTorch SSD300 ({device.upper()})",
        "Avg Preprocess (ms)": avg_pre,
        "Avg Inference (ms)": avg_inf,
        "Avg Postprocess (ms)": avg_post,
        "Total (ms)": total_ms,
        "FPS": fps,
    }



In [8]:
data_yaml = "./Dataset/Detect/data_detect_tt100k/data.yaml"

# Model paths
models_detection = {
    "Faster_RCNN": "./weight/faster_rcnn.pth",
    "SSD300": "./weight/ssd300.pth",
}

results = []

# --- Benchmark Faster R-CNN ---
print("\n" + "="*60)
print("🔥 FASTER R-CNN BENCHMARK")
print("="*60)

try:
    result = benchmark_faster_rcnn_inference(
        model_path=models_detection["Faster_RCNN"],
        data_yaml=data_yaml,
        model_name="Faster_RCNN_ResNet50",
        input_size=(640, 640),
        num_warmup=5,
        num_samples=25,
        conf_threshold=0.25,
        device='cpu'
    )
    results.append(result)
except Exception as e:
    print(f"❌ Error benchmarking Faster R-CNN: {e}")

# --- Benchmark SSD300 ---
print("\n" + "="*60)
print("🔥 SSD300 BENCHMARK")
print("="*60)

try:
    result = benchmark_ssd300_inference(
        model_path=models_detection["SSD300"],
        data_yaml=data_yaml,
        model_name="SSD300_VGG16",
        input_size=(640, 640),
        num_warmup=5,
        num_samples=25,
        conf_threshold=0.25,
        device='cpu'
    )
    results.append(result)
except Exception as e:
    print(f"❌ Error benchmarking SSD300: {e}")

# --- Print Summary ---
print("\n" + "="*60)
print("✅ BENCHMARK SUMMARY")
print("="*60)

df = pd.DataFrame(results)
print(df.to_string(index=False))

# Save to CSV
save_path = "./Eval/detect_eval_results_fps_rcnn_ssd.csv"
df.to_csv(save_path, index=False)
print(f"\n💾 Saved results to: {save_path}")


🔥 FASTER R-CNN BENCHMARK

🚀 Benchmarking Faster R-CNN: faster_rcnn.pth
📐 Input size: (640, 640)
🖥️  Device: cpu
📊 Running 25 samples (after 5 warmup)



🔥 Warmup (5 runs)...

⏱️  Running inference on 25 test images...


Processing images: 100%|██████████| 25/25 [02:22<00:00,  5.68s/it]



📊 Average Timing (over 25 images):
   🧩 Preprocess: 9.40 ms
   ⚙️  Inference : 5592.59 ms
   📦 Postprocess: 0.22 ms
   ⏱️  Total: 5602.21 ms → 0.18 FPS

🔥 SSD300 BENCHMARK

🚀 Benchmarking SSD300: ssd300.pth
📐 Input size: (640, 640)
🖥️  Device: cpu
📊 Running 25 samples (after 5 warmup)

Downloading: "https://download.pytorch.org/models/vgg16_features-amdegroot-88682ab5.pth" to /home/pi5/.cache/torch/hub/checkpoints/vgg16_features-amdegroot-88682ab5.pth


100%|██████████| 528M/528M [02:37<00:00, 3.50MB/s] 


🔥 Warmup (5 runs)...

⏱️  Running inference on 25 test images...


Processing images: 100%|██████████| 25/25 [00:24<00:00,  1.00it/s]


📊 Average Timing (over 25 images):
   🧩 Preprocess: 7.29 ms
   ⚙️  Inference : 860.88 ms
   📦 Postprocess: 0.15 ms
   ⏱️  Total: 868.33 ms → 1.15 FPS

✅ BENCHMARK SUMMARY
               Model                   Backend  Avg Preprocess (ms)  Avg Inference (ms)  Avg Postprocess (ms)  Total (ms)      FPS
Faster_RCNN_ResNet50 PyTorch Faster-RCNN (CPU)             9.396381         5592.592163              0.217390 5602.205935 0.178501
        SSD300_VGG16      PyTorch SSD300 (CPU)             7.292261          860.882187              0.152407  868.326855 1.151640

💾 Saved results to: ./Eval/detect_eval_results_fps_rcnn_ssd.csv
